### Что это?
Мне пришла в голову идея изобразить на полотне графики котировок ценных бумаг и отметить на них новости о компании или её руководителях, или об отрасли, или о стране. Здесь приведена реализация такого построения графиков.  
Думаю, что такой график котировок с отмеченными новостями поможет проследить, какие события какое влияние оказали на стоимость ценных бумаг. Построение нескольких графиков на одном полотне поможет сравнить, как менялась стоимость разных бумаг.  
Кроме того, это может стать основой для более интересного инструмента, который мог бы не только строить графики, но и в какой-то степени помогать в анализе. Но это уже другая история. А пока просто порисуем графики.  

### Постановка задачи и выбор инструментов
Я решил не делать из этого веб-сервис, потому что не захотел решать вопросы с лицензиями на использование данных биржи и новостных порталов. Но кодом всё таки решил поделиться. Поэтому данный документ носит исключительно образовательный характер, а код не предназначен для боевого применения.  
Такая постановка задачи помогла определиться с инструментами. Документ я подготовил в jupyter notebook, потому что он хорошо подходит для образовательных материалов и экспериментов. Если вы захотите утащить это в настоящий проект, пожалуйста, сделайте всё красиво.  
Код написан на python3, наглядность кода поставлена выше, чем практичность.  

### Установка зависимостей
Давайте сразу установим и подключим необходимые пакеты. Чтобы далее не отвлекаться на это.  

In [2]:
# Common
import warnings
warnings.filterwarnings(action='once')

from random import randint

import datetime
import time

In [3]:
# For http requests
import urllib
import requests
import json

In [4]:
# For data processing
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
# For plotting
!pip install plotly==5.4.0 -q
!pip install jupyter-dash -q
!pip install dash_extensions -q

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash_extensions import Purify

     |████████████████████████████████| 25.3 MB 2.9 MB/s 
     |████████████████████████████████| 7.3 MB 6.3 MB/s 
     |████████████████████████████████| 357 kB 59.2 MB/s 
     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 73 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 80 from PyObject
  return f(*args, **kwds)


### Получение данных
Необходимые для построения графиков данные будем получать с других сайтов.  
Некоторые сайты (Мосбиржа) предоставляют API, с ними работать проще. Другие (Новости РБК) не предоставляют открытый API. Но в любом случае получение данных сводится к отправке http запросов.  
Напишем два класса-клиента, первый будет получать информацию о ценных бумагах через API Мосбиржи. Второй - скачивать новости с сайта РБК.  
Оба клиента будут иметь похожий набор настроек, поэтому объединим все настройки в одном объекте конфигурации.

In [6]:
class Config:
  def __init__(self, proxy_url='', debug_level=0):
    """
    Container for all configuration options:
    proxy_url: proxy URL if any is used, specified as http://proxy:port
    debug_level: 0 - no output, 1 - send debug info to stdout
    """
    self.debug_level = debug_level  
    self.proxy_url = proxy_url

Другие общие для всех http-клиентов настройки можно добавлять в этот класс. Затем объект этого класса, хранящий настройки будем передавать в конструкторы клиентов.  
На данный момент клиенты не используют proxy, хотя конфигурация это предусматривает, реализуем это в другой раз.  

In [7]:
config = Config(debug_level=1)  # Detailed log is enabled.

Давайте получим историю по одной бумаге на рынке за интервал дат. Покажу, как можно сделать это, используя API Мосбиржи.  

Про API Мосбиржи можно почитать здесь: https://www.moex.com/a2193  
Также обратите внимание на эту страницу: https://www.moex.com/ru/products/personal  

Чтобы получить историю, нужно сделать такой запрос: http://iss.moex.com/iss/reference/817  
Напишем маленький клиентский класс для работы только с одним этим методом.  

In [8]:
class MoexClient:
  """
  Sends requests to MOEX API.
  """
  TODAY = datetime.date.today()  # for convenience
  ONE_YEAR_AGO = datetime.date.today() - datetime.timedelta(days=365)  # for convenience
  FIVE_YEARS_AGO = datetime.date.today() - datetime.timedelta(days=5*365)  # for convenience

  def __init__(self, config=Config()):
    """
    Construct an object of MoexClient and set the configuration.
    config: configuration object for MoexClient
    """
    self.config = config

  def get_history(self, engine='stock', market='index', board='SNDX', security='IMOEX', from_date=ONE_YEAR_AGO, till_date=TODAY):
    """
    Get history of one security from MOEX.
    engine: One of engines from https://iss.moex.com/iss/engines.xml
    market: One of markets from http://iss.moex.com/iss/reference/42
    board: One of boards from http://iss.moex.com/iss/reference/43
    security: One of securities from http://iss.moex.com/iss/reference/33
    from_date: The date from which to get the data.
    till_date: The date before which to get the data.
    """

    # Preparing a first request
    url = f'http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/boards/{board}/securities/{security}.json'
    response = requests.get(
        url,
        params={'from': str(from_date), 'till': str(till_date)},
    )

    # Sending the first request
    try:
      result = dict(response.json())
      columns = result['history']['columns']
      data = result['history']['data']
    except:
      print("Error!!!")
      print(response.content)
      return None

    if self.config.debug_level == 1:
      print(f'{security}: got {len(data)} rows. Last date: {data[-1][columns.index("TRADEDATE")]}.')

    # Repeats until all data is received.
    got_count = len(data)
    while got_count > 0:
      response = requests.get(
          url,
          params={'from': str(from_date), 'till': str(till_date), 'start': len(data)},
      )
      result = dict(response.json())
      data.extend(result['history']['data'])
      got_count = len(result['history']['data'])

      if self.config.debug_level == 1:
        print(f'{security}: got {len(data)} rows. Last date: {data[-1][columns.index("TRADEDATE")]}.')
    
    # Create dataframe
    df = pd.DataFrame(columns=columns, data=data)
    return df

  def get_imoex_history(self, from_date=ONE_YEAR_AGO, till_date=TODAY):
    """
    Method is just for convenience. Get history of IMOEX.
    from_date: The date from which to get the data.
    till_date: The date before which to get the data.
    """
    return self.get_history(engine='stock', market='index', board='SNDX', security='IMOEX', from_date=from_date, till_date=till_date)

  def get_share_history(self, security='GAZP', from_date=ONE_YEAR_AGO, till_date=TODAY):
    """
    Method is just for convenience. Get history of shares.
    security: Share's code on MOEX.
    from_date: The date from which to get the data.
    till_date: The date before which to get the data.
    """
    return self.get_history(engine='stock', market='shares', board='TQBR', security=security, from_date=from_date, till_date=till_date)


Посмотрим, что получилось.  
Получим историю изменения индекса Мосбиржи за последний год:

In [9]:
moex = MoexClient(config=config)
imoex = moex.get_imoex_history()

IMOEX: got 100 rows. Last date: 2021-05-31.
IMOEX: got 200 rows. Last date: 2021-10-18.
IMOEX: got 257 rows. Last date: 2022-01-07.
IMOEX: got 257 rows. Last date: 2022-01-07.


Получим историю акций Мечела за последний год:

In [10]:
mtlr = moex.get_share_history(security='MTLR')

MTLR: got 100 rows. Last date: 2021-05-31.
MTLR: got 200 rows. Last date: 2021-10-18.
MTLR: got 257 rows. Last date: 2022-01-07.
MTLR: got 257 rows. Last date: 2022-01-07.


Посмотрим как это выглядит:

In [11]:
imoex

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,2021-01-08,Индекс МосБиржи,Индекс МосБиржи,3454.82,3390.23,3474.66,3390.23,1.321056e+11,0,0,2,1.754780e+13,RUB,5.079231e+09,3,None
1,SNDX,IMOEX,2021-01-11,Индекс МосБиржи,Индекс МосБиржи,3482.48,3455.67,3516.90,3436.66,1.432925e+11,0,0,2,1.768832e+13,RUB,5.079231e+09,3,None
2,SNDX,IMOEX,2021-01-12,Индекс МосБиржи,Индекс МосБиржи,3471.65,3494.60,3520.66,3451.87,1.042165e+11,0,0,2,1.763330e+13,RUB,5.079231e+09,3,None
3,SNDX,IMOEX,2021-01-13,Индекс МосБиржи,Индекс МосБиржи,3470.26,3487.92,3504.56,3448.94,9.961662e+10,0,0,2,1.762627e+13,RUB,5.079231e+09,3,None
4,SNDX,IMOEX,2021-01-14,Индекс МосБиржи,Индекс МосБиржи,3490.85,3461.83,3507.71,3439.01,1.085408e+11,0,0,2,1.773086e+13,RUB,5.079231e+09,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,SNDX,IMOEX,2022-01-03,Индекс МосБиржи,Индекс МосБиржи,3852.50,3825.30,3866.15,3823.50,5.247287e+10,0,0,2,1.905000e+13,RUB,4.944847e+09,3,None
253,SNDX,IMOEX,2022-01-04,Индекс МосБиржи,Индекс МосБиржи,3873.49,3880.20,3892.33,3846.81,6.262441e+10,0,0,2,1.915380e+13,RUB,4.944847e+09,3,None
254,SNDX,IMOEX,2022-01-05,Индекс МосБиржи,Индекс МосБиржи,3815.05,3857.90,3875.25,3805.06,5.984228e+10,0,0,2,1.886482e+13,RUB,4.944847e+09,3,None
255,SNDX,IMOEX,2022-01-06,Индекс МосБиржи,Индекс МосБиржи,3753.29,3758.68,3784.17,3720.17,8.346910e+10,0,0,2,1.855942e+13,RUB,4.944847e+09,3,None


In [12]:
mtlr

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,WAPRICE,CLOSE,VOLUME,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION
0,TQBR,2021-01-08,Мечел ао,MTLR,14685,3.666867e+08,77.86,77.10,78.88,77.50,77.94,77.50,4704901,77.94,77.94,77.50,3.666867e+08,3.666867e+08,3.666867e+08,0,3
1,TQBR,2021-01-11,Мечел ао,MTLR,24719,4.236603e+08,77.50,76.17,79.95,79.21,78.70,79.21,5382949,78.70,78.70,79.21,4.236603e+08,4.236603e+08,4.236603e+08,0,3
2,TQBR,2021-01-12,Мечел ао,MTLR,11688,1.985035e+08,79.21,77.83,79.98,78.20,78.62,78.20,2524883,78.62,78.62,78.20,1.985035e+08,1.985035e+08,1.985035e+08,0,3
3,TQBR,2021-01-13,Мечел ао,MTLR,46097,9.325158e+08,78.70,77.20,84.14,84.05,81.30,84.05,11470056,81.30,81.30,84.05,9.325158e+08,9.325158e+08,9.325158e+08,0,3
4,TQBR,2021-01-14,Мечел ао,MTLR,80915,1.326343e+09,84.20,81.28,86.47,81.75,83.51,81.75,15881765,83.51,83.51,81.75,1.326343e+09,1.326343e+09,1.326343e+09,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,TQBR,2022-01-03,Мечел ао,MTLR,22032,6.355210e+08,125.35,124.69,128.51,127.50,127.17,127.50,4997306,127.17,127.17,127.50,6.355210e+08,6.355210e+08,6.355210e+08,0,3
253,TQBR,2022-01-04,Мечел ао,MTLR,16027,3.541721e+08,127.97,125.50,130.00,127.20,127.39,127.20,2780269,127.39,127.39,127.20,3.541721e+08,3.541721e+08,3.541721e+08,0,3
254,TQBR,2022-01-05,Мечел ао,MTLR,16050,3.793101e+08,126.49,122.73,127.89,124.73,125.34,124.73,3026127,125.35,125.35,124.73,3.793101e+08,3.793101e+08,3.793101e+08,0,3
255,TQBR,2022-01-06,Мечел ао,MTLR,20491,3.745795e+08,122.00,120.00,124.40,122.40,122.32,122.40,3062338,122.32,122.32,122.40,3.745795e+08,3.745795e+08,3.745795e+08,0,3


Отлично.  
Теперь нужно научиться получать новости по определённому запросу.  
Покажу, как можно получить новости с сайта РБК.  
Если вы захотите воспользоваться этим кодом в любых целях, то ознакомьтесь с информацией на странице https://www.rbc.ru/privacy/  

In [13]:
class RbcClient:
  """
  Sends requests to rbc.ru
  """
  TODAY = datetime.date.today()  # for convenience
  ONE_YEAR_AGO = datetime.date.today() - datetime.timedelta(days=365)  # for convenience
  FIVE_YEARS_AGO = datetime.date.today() - datetime.timedelta(days=5*365)  # for convenience

  def __init__(self, config=Config()):
    """
    Construct an object of RbcClient and set the configuration.
    config: configuration object for RbcClient
    """
    self.config = config

  def download_news(self, query='Газпром', from_date=ONE_YEAR_AGO, till_date=TODAY):
    """
    Get news by query from rbc.ru
    query: Keywords for search.
    from_date: The date from which to get the news.
    till_date: The date before which to get the news.
    """

    session = requests.Session()  # Storage for cookies
    limit = 30  # News per request
    got = limit  # Count of downloaded news last time

    # Preparing
    general_url = f'https://www.rbc.ru/search/?query={urllib.parse.quote_plus(query.lower())}&dateFrom={from_date.strftime("%d.%m.%Y")}&dateTo={till_date.strftime("%d.%m.%Y")}'
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive',
        'Host': 'www.rbc.ru',
        'Referer': general_url,
        'TE': 'Trailers',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'X-Requested-With': 'XMLHttpRequest',
    }
    session.get(general_url, headers=headers)

    # Downloads until all news are received.
    items = []
    while got == limit:
      offset = len(items)
      time.sleep(3)  # For not to overload the service
      url = f'https://www.rbc.ru/v10/search/ajax/?dateFrom={from_date.strftime("%d.%m.%Y")}&dateTo={till_date.strftime("%d.%m.%Y")}&offset={offset}&limit={limit}&query={query.lower()}'
      response = session.get(
          url,
          headers=headers,
      )
      got = len(response.json()['items'])
      items.extend(response.json()['items'])
      if self.config.debug_level == 1:
        print(f"{query}: got {len(items)} news. Last date: {items[-1]['publish_date']}.")

    # Postprocessing and creating dataframe
    for item in items:
      item['photo'] = item['photo']['url']
      if isinstance(item['authors'], list):
        item['authors'] = ', '.join(item['authors'])
    df_news = pd.DataFrame(columns=list(items[0].keys()), data=items)
    df_news.drop_duplicates(subset='title', keep='first', inplace=True, ignore_index=False)
    return df_news

Проверим работоспособность:

In [14]:
rbc = RbcClient(config=config)
news_mtlr = rbc.download_news("Мечел")

Мечел: got 30 news. Last date: Tue, 21 Sep 2021 15:39:17 +0300.
Мечел: got 60 news. Last date: Wed, 02 Jun 2021 06:00:28 +0300.
Мечел: got 85 news. Last date: Tue, 12 Jan 2021 07:30:19 +0300.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ResourceWarning: unclosed <ssl.SSLSocket fd=62, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 58734), raddr=('185.72.229.3', 443)>
  


In [15]:
news_mtlr

,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons
0,61cc4ee39a79470a036a29cb,https://quote.rbc.ru/news/article/61cc4ee39a79...,1641276000,"Tue, 04 Jan 2022 09:00:00 +0300",Эти 15 акций выросли больше всего с начала пан...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,Инвестиции,None,None,Алексей Митраков,... списке пять бумаг металлургов — акции «А...
1,61c5882b9a79474c404bd73a,https://www.rbc.ru/finances/29/12/2021/61c5882...,1640754015,"Wed, 29 Dec 2021 08:00:15 +0300",Аналитики назвали самые доходные инвестиции по...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,None,Финансы,None,Маргарита Мордовина,... компаний. Лидером по доходности в уходящ...
2,61c07be29a79478538479018,https://pro.rbc.ru/news/61c07be29a79478538479018,1640677001,"Tue, 28 Dec 2021 10:36:41 +0300",2021-й — супергод для сталеваров. Что ждет их ...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,РБК Pro,None,None,None,"... %, у НЛМК – 23%, а у Evraz Group – 12%. О..."
3,61c44c5c9a79475ffe9a8997,https://quote.rbc.ru/news/article/61c44c5c9a79...,1640324420,"Fri, 24 Dec 2021 08:40:20 +0300",Самые недооцененные акции России и США. Что ку...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,Инвестиции,None,None,Алексей Митраков,"... и Приволжье, «Сургутнефтегаз» и ФСК ЕЭС...."
4,61c41fe89a79474a1873b62b,https://quote.rbc.ru/news/short_article/61c41f...,1640245115,"Thu, 23 Dec 2021 10:38:35 +0300",«Мечел» может перезапустить свой никелевый ком...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,Инвестиции,None,None,Марина Ануфриева,... <b>Мечел</b>» может перезапустить никелев...
...,...,...,...,...,...,...,...,...,...,...,...
80,6013c3b09a79472b8f5009b8,https://www.rbc.ru/economics/29/01/2021/6013c3...,1611910583,"Fri, 29 Jan 2021 11:56:23 +0300",Лисин решил написать Мишустину о «непродуманны...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,None,Экономика,None,Евгений Калюков,"... , говорится на сайте организации. Членами..."
81,601006b39a79477a91103762,https://www.rbc.ru/business/26/01/2021/601006b...,1611676906,"Tue, 26 Jan 2021 19:01:46 +0300",Наследники Босова продали «Ростеху» и партнера...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,None,Бизнес,None,"Светлана Бурмистрова, Тимофей Дзядко",... Огоджа. [ РБК ] Основатель Yota стал пре...
82,6001a5349a7947455a1c88c8,https://www.rbc.ru/finances/19/01/2021/6001a53...,1611039610,"Tue, 19 Jan 2021 10:00:10 +0300",Сделка со Сбербанком не дала рынку слияний упа...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,None,Финансы,None,Павел Казарновский,... Альберта Авдоляна проекта по разработке ...
83,600553629a794721b88d2867,https://trends.rbc.ru/trends/industry/60055362...,1610967616,"Mon, 18 Jan 2021 14:00:16 +0300",Как Rescore научила алгоритмы проверять контра...,https://s0.rbk.ru/v6_top_pics/resized/250xH/me...,Тренды,Индустрия 4.0,None,Юлия Макарова,"... , который ранее руководил ИТ-проектами в ..."


Хорошо. Перейдём к отрисовке графиков.

### Рисуем графики
Сначала я пробовал рисовать, используя matplotlib. Но оказалось довольно сложно сделать интересный dashboard без высокоуровневых инструментов.  
Поэтому сразу воспользуемся связкой plotly + dash.  
Это отличные инструменты для красивой отрисовки и создания функциональных дашбордов.  
Подробнее:  
https://plotly.com  
https://dash.plotly.com  



Чтобы познакомиться с plotly, посмотрите, как легко с его помощью нарисовать красивый график:  

In [16]:
def draw_history(df, y_column='CLOSE'):
  """
  Plot history of security received from MOEX API.
  y_column: The name of column which contain target value. Default is 'CLOSE', the price at the close of trading.
  """
  x_column = 'TRADEDATE'  # The name of column with dates.
  title = f"Value of {df['SECID'].iloc[0]} ({df[x_column].iloc[0]} - {df[x_column].iloc[-1]})"
  fig = px.line(df, x=x_column, y=y_column, title=title,
                labels={x_column: "Date", y_column: f"Value of {df['SECID'].iloc[0]}"})
  fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
  )
  fig.show()

In [17]:
draw_history(imoex)

/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working



In [18]:
draw_history(mtlr)

Но нам нужен больший функционал: давайте сделаем интерактивный график. Пусть на графике будут отмечены новости о компании, а при нажатии на отметку появляется дополнительная информация о новости. Для этого нам понадобится dash.  

Новости на графике отметим точками. А цвет точки будет зависеть от категории новости.  
Категория в новостях закодирована двумя столбцами: project и category. Нужно будет сделать LabelEncoding по этим двум столбцам.  
Цвет в RGB кодируется тремя байтами, поэтому цвета разных категорий сгенерируем случайно как трёхбайтные числа.  
Чтобы работать с цветами в таком представлении, определим вспомогательные функции:

In [25]:
def generate_colors(count):
  """
  Generates many random colors as integer numbers. Saves them in dict {No:Color}.
  count: count of generated colors
  """
  colors = dict()
  for num in range(count):
    colors[num] = randint(0, 0xFFFFFF)
  return colors

def int_to_rgb(color_int):
  """
  Convert color from integer to tuple (r, g, b).
  color_int: Color as integer number.
  """
  blue =  color_int & 255
  green = (color_int >> 8) & 255
  red =   (color_int >> 16) & 255
  return red, green, blue

def rgb_to_int(color_rgb):
  """
  Convert color from tuple to integer.
  color_rgb: Color as tuple (r, g, b).
  """
  red = color_rgb[0]
  green = color_rgb[1]
  blue = color_rgb[2]
  color_int = (red << 16) + (green << 8) + blue
  return color_int

def mix_colors(colors_int, alpha=0.7):
  """
  Mixes components of several colors, adds an alpha channel and serializes as a string 'rgba(r, g, b, a)'.
  colors_int: list of colors
  alpha: value of alpha channel
  """
  r_avg = 0
  g_avg = 0
  b_avg = 0

  for color_int in colors_int:
    r, g, b = int_to_rgb(color_int)
    r_avg += r
    g_avg += g
    b_avg += b
  
  r_avg /= len(colors_int)
  g_avg /= len(colors_int)
  b_avg /= len(colors_int)

  return f"rgba({int(r_avg)}, {int(g_avg)}, {int(b_avg)}, {alpha})"

А теперь напишем код для создания дашборда:

In [26]:
def draw_histories_dash(quotations, news, y_column='CLOSE', normalize_type="max"):
  """
  Plots the quotations and news on same canvas and connect handler for clicking on a marker.
  quotations: list of dataframes gotten by MoexClient
  news: list of dataframes gotten by RbcClient or None values
  y_column: The name of column which contain target value. Default is 'CLOSE', the price at the close of trading.
  normalize_type: a method of normalization ("max", "minmax", "none") for quotations. To ensure the same scale of different graphs.
  """
  assert(len(quotations) == len(news))  # One news dataframe (may be None) for each security
  x_column = 'TRADEDATE'  # The name of column with dates.

  # Prepares title of plot
  companies = [df['SECID'].iloc[0] for df in quotations]
  title = f"Normalized values of {', '.join(companies)} ({quotations[0][x_column].iloc[0]} - {quotations[0][x_column].iloc[-1]})"

  # Normalizes quotations
  norm_column = 'NORM'  # The name of new column with normalized values.
  quotations_norm = [x.copy() for x in quotations]
  for df in quotations_norm:
    if normalize_type == "max":
      df[norm_column] = df[y_column] / df[y_column].max()
    elif normalize_type == "minmax":
      df[norm_column] = (df[y_column] - df[y_column].min()) / df[y_column].max()
    else:
      title = f"Values of {', '.join(companies)} ({quotations[0][x_column].iloc[0]} - {quotations[0][x_column].iloc[-1]})"
      df[norm_column] = df[y_column]
  
  # Prepares news for drawing
  compact_news = []
  for i, df_original in enumerate(news):  # for each news dataframe
    if df_original is None:
      compact_news.append(None)
      continue
    
    df = df_original.copy()  # in order not to spoil the original dataframe

    # converts dates of news to same format like quotations
    df[x_column] = df['publish_date_t'].astype('int64').apply(lambda x: str(datetime.datetime.utcfromtimestamp(x).date()))
    
    # label encoding for project and category columns
    def join_categories(categories):
      cleaned_cats = [str(cat) for cat in categories if cat is not None]
      if len(cleaned_cats) == 0:
        return "Прочее"
      return ', '.join(cleaned_cats)
    df['category'] = news_mtlr[['project', 'category']].apply(lambda row: join_categories(row.values), axis=1)
    cats = LabelEncoder().fit_transform(df['category'])

    # generates color for each category and saves it in dataframe
    colors = generate_colors(cats.max() + 1)
    df['color'] = [colors[cat] for cat in cats]

    # groups news by date and filters columns
    grouped = df.groupby(x_column)
    interesting_columns = ['title', 'anons', 'fronturl', 'color', 'category']
    compact_df = grouped[interesting_columns[0]].apply(list).reset_index(name=interesting_columns[0])
    for col in interesting_columns[1:]:
      compact_df[col] = grouped[col].apply(list).reset_index(name=col)[col]
    
    # Mixes colors for different categories in one day
    compact_df['color'] = compact_df['color'].apply(mix_colors)

    # Matches quotations and news
    interdates = pd.Series(sorted(list(set(quotations_norm[i][x_column]).intersection(set(compact_df[x_column])))))
    compact_df.loc[compact_df[x_column].isin(interdates), y_column] = quotations_norm[i].loc[quotations_norm[i][x_column].isin(interdates), y_column].values
    compact_df[y_column].fillna(method='ffill', inplace=True)
    compact_df.loc[compact_df[x_column].isin(interdates), norm_column] = quotations_norm[i].loc[quotations_norm[i][x_column].isin(interdates), norm_column].values
    compact_df[norm_column].fillna(method='ffill', inplace=True)

    # Makes short descriptions for points
    hovers = []
    for _, row in compact_df.iterrows():
      hovers.append(f"True value: {row[y_column]}<br><br>" + "<br>".join(row['title']))
    compact_df['hovertext'] = hovers

    # Makes full descriptions for points
    fulls = []
    for _, row in compact_df.iterrows():
      title_anons_link_cat = zip(row['title'], row['anons'], row['fronturl'], row['category'])
      fulls.append("<br><br>".join([f"<p><a href='{l}'>{t}</a><br>Категория: {c}<br>{a}</p>" for t, a, l, c in title_anons_link_cat]))
    compact_df['fulltext'] = fulls

    # Done, saves it
    compact_news.append(compact_df)

  # Creates a canvas
  fig = go.Figure()

  # Draws quotations
  for df in quotations_norm:
    fig.add_trace(go.Scatter(x=df[x_column], y=df[norm_column],
                    mode='lines',
                    name=df['SECID'].iloc[0],
                    hovertext=df[y_column].astype('str').apply(lambda x: f"True value: {x}").values))
  
  # Draws news
  for df in compact_news:
    if df is not None:
      fig.add_trace(
          go.Scatter(x=df[x_column], 
                     y=df[norm_column], 
                     mode='markers', 
                     name=f"News about {quotations_norm[i]['SECID'].iloc[0]}", 
                     hovertext=df['hovertext'].values, 
                     customdata=df['fulltext'].values,
                     marker=dict(color=df['color'].values, size=9)))

  # Configures plot
  fig.update_layout(
    title=title,
    xaxis_title="Date",
    yaxis_title="Normalized value",
    legend_title="Legend",
  )
  fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
  )

  # Creates dashboard
  app = JupyterDash("Quotations")

  # Configures an interface of dashboard
  app.layout = html.Div([
      dcc.Graph(
          id='basic-interactions',
          figure=fig
      ),
      html.Div([Purify(id='click-data')], className='row'),
  ])

  # Connects handler for click on news
  @app.callback(
      Output('click-data', 'html'),
      Input('basic-interactions', 'clickData'))
  def display_click_data(clickData):
      return clickData['points'][0]['customdata']

  # Done, runs dashborad and shows it
  app.run_server(mode='inline', host='localhost', port=1050, )


In [27]:
draw_histories_dash([imoex, mtlr], [None, news_mtlr])

<IPython.core.display.Javascript object>

Отлично, у нас всё получилось.  
Если будут идеи и замечания, создавайте issue в репозитории на github.  